# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
import warnings

sns.set_theme(style="darkgrid")
warnings.filterwarnings('ignore')

# 2. Data Preprocessing

In [ ]:
df = pd.read_csv('data/FT1000_v0.csv')
df.head()

In [ ]:
(df.keys().str[:7] == 'Unnamed').sum()

In [ ]:
(df.iloc[:, 13:].isnull().sum() == 1001).sum()

Trước khi tiến hành các thao tác tiền xử lí chính, nhóm nhận thấy data thu thập được có các cột Unnamed chứa toàn các dữ liệu NaN, do đó nhóm sẽ loại bỏ các cột này

In [ ]:
df.drop(df.iloc[:, 13:], axis = 1, inplace = True)

## 2.1. Tổng quan dữ liệu
Số cột và số dòng của dữ liệu

In [ ]:
df.shape

Ý nghĩa của từng cột:
- `Rank`: thứ hạng của công ty
- `Name`: tên công ty
- `in 2021 ranking`: Công ty có nằm trong bảng xếp hạng năm 2021 hay không
- `in 2020 ranking`: Công ty có nằm trong bảng xếp hạng năm 2020 hay không
- `Country`: công ty thuộc về quốc gia nào
- `Sector`: lĩnh vực hoạt động của công ty
- `Absolute Growth Rate %`: Tỉ lệ tăng trưởng tuyệt đối từ 2017 tới 2020
- `Compound Annual Growth Rate (CAGR) %`: Tỉ lệ tăng trưởng kép từ 2017 tới 2020
- `Revenue 2020 (€)`: Doanh thu năm 2020 tính theo đồng Euro
- `Revenue 2017 (€)`: Doanh thu năm 2017 tính theo đồng Euro
- `Number of employees 2020`: Số lượng nhân viên năm 2020
- `Number of employees 2017`: Số lượng nhân viên năm 2017
- `Founding Year`: Năm công ty thành lập

Công thức tính 2 cột `Absolute Growth Rate %` và `Compound Annual Growth Rate (CAGR) %` đã được đề cập ở notebook `Data Collecting`, ở đây nhóm chỉ giải thích sự khác nhau về mặt ý nghĩa của 2 cột. 2 thang đo này thường được dùng để đánh giá kết quả đầu tư, trong trường hợp của dữ liệu này, là sự tăng trưởng của các công ty
- <t style = "color:green">Absolute Growth Rate</t> hay <t style = "color:green">tỉ lệ tăng trưởng tuyệt đối</t>, đo độ tăng trưởng của công ty trong một giai đoạn thời gian
- <t style = "color:blue">Compound Annual Growth Rate (CAGR)</t> hay <t style = "color:blue">tỉ lệ tăng trưởng kép</t>, đo độ tăng trưởng của công ty trong từng năm trong một giai đoạn thời gian. <t style = "color:blue">Tỉ lệ tăng trưởng kép</t> được tính dựa trên sự tái đầu tư của doanh thu mỗi năm, cho thấy tốc độ tăng trưởng của công ty trong từng năm. Còn <t style = "color:green">tỉ lệ tăng trưởng tuyệt đối</t> chỉ cho thấy sự tăng trưởng của công ty trong cả quá trình.

## 2.2. Data types
Kiểm tra kiểu dữ liệu của các cột

In [ ]:
def open_object_dtype(s):
    dtypes = set(s.apply(lambda item : type(item)))
    return dtypes

df.apply(open_object_dtype)

Quan sát kiểu dữ liệu của các cột, nhóm có nhận xét như sau:
- Các cột có kiểu dữ liệu hợp lí:
    - Numerical: `Absolute Growth Rate (%)` và `Compound Annual Growth Rate (CAGR) %`
    - Categorical: `Name, in 2021 ranking, in 2020 ranking, Country, Sector`
- Các cột dữ liệu float cần chuyển sang int: `Rank, Revenue 2020 (€), Revenue 2017 (€), Number of employees 2020, Founding Year`
- Cột có kiểu dữ liệu không rõ ràng: `Number of employees 2017`, gồm str và float. Vì ý nghĩa của cột này là số lượng nhân viên của công ty trong năm 2017, (tương tự cột `Number of employees 2020`), nhóm sẽ chuyển cột này thành int

Tuy nhiên việc chuyển kiểu dữ liệu sẽ được thực hiện cùng với việc loại bỏ missing values bên dưới

## 2.3. Missing values
Kiểm tra missing values trên tất cả các cột

In [ ]:
df.isnull().sum()

In [ ]:
df.iloc[1000,:]

In [ ]:
df.drop(1000, axis = 0, inplace = True)

Đa số các cột có 1 giá trị NaN, để ý thấy hàng có index 1000 của df chứa toàn các giá trị NaN, nhóm quyết định drop hàng này. Do đó chỉ còn 2 cột có một vài giá trị khuyết đó là `Number of employees 2020` và `Number of employees 2017`

In [ ]:
df[df.isnull().any(axis = 1)]

In [ ]:
df['Number of employees 2020'].plot(kind = 'box')

Quan sát thấy phân bố của cột `Number of employees 2020` bị skew quá lớn, do đó nhóm quyết định sử dụng giá trị trung vị của cột này để điền vào các giá trị khuyết

In [ ]:
df['Number of employees 2020'].fillna(df['Number of employees 2020'].median(), inplace = True)

Với trường hợp của cột `Number of employees 2020`, ta thấy cột này có cả kiểu dữ liệu str, trước khi tiến hành xử lí các missing value, ta sẽ đi tìm hiểu các dòng chứa str trong cột này

In [ ]:
df[[isinstance(value, str) for value in df['Number of employees 2017']]]['Number of employees 2017']

In [ ]:
(df['Number of employees 2017'] == 'na').sum()

Cột này ngoài chứa các giá trị số ở dạng str còn chứa str `'na'`, ta thay các giá trị này bằng `np.nan`

In [ ]:
df.replace('na', np.nan, inplace = True)

In [ ]:
df['Number of employees 2017'] = df['Number of employees 2017'].astype(float)
df['Number of employees 2017'].fillna(df['Number of employees 2017'].median(), inplace = True)

In [ ]:
df.isnull().sum()

Sau khi đã xử lí hết các missing value, ta chuyển đổi kiểu dữ liệu của các cột cho phù hợp

In [ ]:
df.iloc[:, [0, 8, 9, 10, 11, 12]] = df.iloc[:, [0, 8, 9, 10, 11, 12]].astype(int)

## 2.3. Missing values
Kiểm tra missing values trên tất cả các cột

In [ ]:
df.isnull().sum()

In [ ]:
df.iloc[1000,:]

In [ ]:
df.drop(1000, axis = 0, inplace = True)

Đa số các cột có 1 giá trị NaN, để ý thấy hàng có index 1000 của df chứa toàn các giá trị NaN, nhóm quyết định drop hàng này. Do đó chỉ còn 2 cột có một vài giá trị khuyết đó là `Number of employees 2020` và `Number of employees 2017`

In [ ]:
df[df.isnull().any(axis = 1)]

In [ ]:
df['Number of employees 2020'].plot(kind = 'box')

Quan sát thấy phân bố của cột `Number of employees 2020` bị skew quá lớn, do đó nhóm quyết định sử dụng giá trị trung vị của cột này để điền vào các giá trị khuyết

In [ ]:
df['Number of employees 2020'].fillna(df['Number of employees 2020'].median(), inplace = True)

Với trường hợp của cột `Number of employees 2020`, ta thấy cột này có cả kiểu dữ liệu str, trước khi tiến hành xử lí các missing value, ta sẽ đi tìm hiểu các dòng chứa str trong cột này

In [ ]:
df[[isinstance(value, str) for value in df['Number of employees 2017']]]['Number of employees 2017']

In [ ]:
(df['Number of employees 2017'] == 'na').sum()

Cột này ngoài chứa các giá trị số ở dạng str còn chứa str `'na'`, ta thay các giá trị này bằng `np.nan`

In [ ]:
df.replace('na', np.nan, inplace = True)

In [ ]:
df['Number of employees 2017'] = df['Number of employees 2017'].astype(float)
df['Number of employees 2017'].fillna(df['Number of employees 2017'].median(), inplace = True)

In [ ]:
df.isnull().sum()

Sau khi đã xử lí hết các missing value, ta chuyển đổi kiểu dữ liệu của các cột cho phù hợp

In [ ]:
df.iloc[:, [0, 8, 9, 10, 11, 12]] = df.iloc[:, [0, 8, 9, 10, 11, 12]].astype(int)

## 2.4. Duplicated rows
Kiểm tra xem dữ liệu có dòng nào bị trùng hay không

In [ ]:
df.duplicated().sum()

Ngoài việc kiểm tra từng dòng trên tất cả các cột, ta còn cần kiểm tra giá trị lặp trên cột `Rank` và `Name` vì 2 cột này được xem như key value, mỗi `Rank` phải nằm trong khoảng từ 1 tơi 1000 và không được trùng nhau, mỗi công ty chỉ được xếp hạng đúng một lần

In [ ]:
df['Rank'].duplicated().sum()

In [ ]:
((df['Rank'] <= 1000) & (df['Rank'] >= 1)).sum()

In [ ]:
df['Name'].duplicated().sum()